# Proyecto MusicStream – Años 2010- 2020

Extracción de datos desde Spotify y Last.fm

## 1. Importaciones y configuración de librerías

In [ ]:
# ===============================
# LIBRERÍAS iNCORPORADAS EN PYTHON
# ===============================

import os                     
# Permite a Python comunicarse con el sistema operativo
# Lo usamos para leer variables guardadas fuera del código

import time                  
# Permite hacer pausas entre peticiones
# Control de ritmo → evitar límites de la API (rate limits)

from urllib.parse import quote   
# Convierte textos con espacios o acentos en un formato válido para URLs
# Ej: "Red Hot Chili Peppers" → "Red%20Hot%20Chili%20Peppers"

# ===============================
# LIBRERÍAS EXTERNAS (instalar con pip install)
# ===============================
from dotenv import load_dotenv 
# Carga un archivo .env donde guardamos datos sensibles
# (contraseñas, claves de API) sin escribirlos directamente en el código

import requests              
# Librería general para hacer peticiones a APIs que no tienen librería propia

import spotipy               
# Librería que nos facilita hablar con la API de Spotify
# Evita que tengamos que construir peticiones HTTP a mano

from spotipy.oauth2 import SpotifyClientCredentials  
# Se usa para identificarnos ante Spotify usando credenciales
# Es como mostrar un carnet para que Spotify sepa quién hace la petición

from spotipy.exceptions import SpotifyException       
# Nos permite detectar y manejar errores cuando Spotify no responde bien

import pandas as pd          
# Herramienta principal para trabajar con tablas de datos (DataFrames)

import numpy as np           
# Ayuda a manejar valores vacíos (NaN) y tipos compatibles con bases de datos

# ===============================
# CONEXIÓN A BASE DE DATOS
# ===============================
import mysql.connector      
# Permite conectar Python con una base de datos MySQL

from mysql.connector import Error  
# Sirve para capturar errores y evitar que el programa se rompa sin avisar

# ===============================
# CARGA DE VARIABLES DE ENTORNO
# ===============================
load_dotenv()  
# Lee el archivo .env y carga sus valores en el entorno del sistema

MYSQL_HOST = os.getenv("MYSQL_HOST")        
# Dirección del servidor donde está la base de datos

MYSQL_USER = os.getenv("MYSQL_USER")        
# Usuario con permiso para acceder a la base de datos

MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")
# Contraseña de la base de datos (nunca debe escribirse directamente en el código)


## 2. Credenciales y conexión Spotify

In [ ]:
# --- Paso 1: Cómo obtener Client ID y Client Secret (Spotify) y qué es OAuth ---
# Usamos OAuth (Open Authorization) para poder acceder a datos públicos de Spotify (artistas, álbumes, playlists públicas)

# --- Paso 2: Cómo obtener Client ID y Client Secret (Spotify) ---
# 1️⃣ Ve a https://developer.spotify.com/dashboard e inicia sesión
# 2️⃣ Haz clic en "Create an App"
# 3️⃣ Completa los campos:
#    - App name: ej. "Mi Analizador de Música"
#    - App description: ej. "Proyecto educativo para analizar datos de canciones"
#    - Marca las casillas según cómo usarás la API (ej. "I only use the Web API")
# 4️⃣ Haz clic en "Create"
# 5️⃣ Copia el Client ID y haz clic en "SHOW CLIENT SECRET" para ver el secreto
# 6️⃣ Guárdalos en un archivo llamado ".env" en la raíz del proyecto:
#    SPOTIPY_CLIENT_ID=tu_id_aquí
#    SPOTIPY_CLIENT_SECRET=tu_secreto_aquí
# 7️⃣ CREA UN ARCHIVO LLAMADO ".gitignore" EN LA RAÍZ DEL PROYECTO Y AÑADE:
#    .env
#    Esto evita que Git suba tus credenciales a GitHub por accidente

# --- Paso 3: Recuperar las credenciales desde el archivo .env ---
client_id = os.getenv("SPOTIFY_CLIENT_ID")          # Identificador único de tu app
client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")  # Clave secreta de tu app

# --- Paso 3: Validación de credenciales ---
if not client_id or not client_secret:
    raise ValueError(
        "❌ Las credenciales de Spotify no están cargadas.\n"
        "Sigue los pasos detallados arriba para obtener Client ID y Client Secret "
        "y guardarlos en tu archivo .env."
    )

# --- Paso 4: Inicialización ---
# Objetivo:
#   - Detectar si hay algún problema con las credenciales

try:
    # usamos las credenciales que permiten a nuestra app acceder a Spotify
    mis_credenciales = SpotifyClientCredentials(
        client_id=client_id,
        client_secret=client_secret
    )
    
    # Cada consulta a la API usará automáticamente nuestras credenciales
    spotify = spotipy.Spotify(auth_manager=mis_credenciales)
    
    # Confirmación visual de que todo está listo
    print("✅ Conexión con Spotify inicializada correctamente.")
    print("🔹 Listo para buscar artistas, álbumes y playlists públicas.")

except spotipy.exceptions.SpotifyException as e:
    # Si falla la conexión o las credenciales son incorrectas, mostramos el error
    raise ConnectionError(f"❌ Error al conectar con Spotify: {e}")


✅ Conexión con Spotify inicializada correctamente.
🔹 Listo para buscar artistas, álbumes y playlists públicas.


In [ ]:
# ===============================
# FLUJO PROFESIONAL DETALLADO: SPOTIFY API
# ===============================

# OBJETIVO GENERAL:
# Aprender a usar la API de Spotify como lo haría un/a analista de datos:
#
#   1️⃣ Entender cómo Spotify organiza la información
#   2️⃣ Usar SEARCH solo para descubrir IDs
#   3️⃣ Usar endpoints específicos con esos IDs
#
# Idea clave:
# 👉 SEARCH = herramienta de descubrimiento
# 👉 ENDPOINTS = extracción real de datos para análisis

# ------------------------------------------------
# 1️⃣ ENTENDER LA API DE SPOTIFY (MODELO MENTAL)
# ------------------------------------------------
# Spotify NO es un buscador libre como Google.
# Es una base de datos estructurada con relaciones claras.
#
# Los principales recursos son:
#   - Artistas
#   - Álbumes / Singles
#   - Canciones (tracks)
#   - Playlists
#
# Relación jerárquica principal:
#
#   ARTISTA
#      ↓
#   ÁLBUM
#      ↓
#   CANCIONES
#
# Cada recurso tiene un ID único que Spotify usa internamente.
# Ejemplos reales:
#
#   Artista  → Shakira  → '0TnOYISbd1XYRBk9myaseg'
#   Álbum    → El Dorado → '3jjujdWJ72nww5eGnfs2E7'
#   Canción  → Chantaje → '0k17h0D3J5VfsdmQ1iZtE9'
#
# PROBLEMA REAL:
# 👉 Normalmente NO conocemos estos IDs al empezar
#
# SOLUCIÓN:
# 👉 Usamos search() SOLO para obtener esos IDs

# ------------------------------------------------
# 2️⃣ SEARCH(): DESCUBRIR IDs Y FILTRAR RESULTADOS
# ------------------------------------------------
# search() es el punto de entrada más común cuando:
#   - Tenemos texto (nombre de artista, álbum, canción)
#   - NO tenemos un ID todavía
#
# El parámetro más importante es `q`
#
# Dentro de `q` podemos usar filtros oficiales de Spotify:
#
#   album       → nombre del álbum
#   artist      → nombre del artista
#   track       → nombre de la canción
#   year        → año o rango (ej. 2010 o 2010-2015)
#   genre       → género musical (latin, jazz, rock, etc.)
#   upc         → identificador comercial de álbum
#   isrc        → identificador internacional de canción
#   tag:hipster → contenido alternativo / menos comercial
#   tag:new     → lanzamientos recientes
#
# IMPORTANTE:
# `q` puede combinar filtros:
#   q='artist:Shakira year:2010'


# Además de `q`, search() acepta parámetros de control:
#
#   type   → qué tipo de recurso queremos:
#            'artist', 'album', 'track', 'playlist'
#
#   market → país de referencia (US, ES, MX...)
#            afecta disponibilidad y popularidad
#
#   limit  → cuántos resultados devuelve (1–50)
#            para análisis solemos usar pocos y luego paginar
#
#   offset → desde qué posición empezar
#            se usa para recorrer resultados grandes
#
#   include_external → incluir contenido externo (normalmente NO se usa)
#
# En la práctica profesional:
# 👉 SEARCH se usa para localizar el recurso correcto
# 👉 Luego se abandona y se usan endpoints directos


# ------------------------------------------------
# 3️⃣ OBTENER EL ID Y DATOS DEL ARTISTA
# ------------------------------------------------
# CASO REAL:
# "Quiero analizar a un artista, pero solo conozco su nombre"

nombre_artista = "Shakira"  # Puede ser cualquier artista

resultado = spotify.search(
    q=f'artist:{nombre_artista}',  # Filtro principal: nombre del artista
    type='artist',                 # Buscamos SOLO artistas
    market='US',                   # Mercado de referencia
    limit=1,                       # Solo el resultado más relevante
    offset=0                       # Inicio de resultados
)

# Extraemos el primer artista encontrado
# (Spotify ya ordena por relevancia)
artista = resultado['artists']['items'][0]

# ID único del artista (CLAVE para siguientes pasos)
artist_id = artista['id']

# ------------------------------------------------
# 4️⃣ INTERPRETAR LOS DATOS DEL ARTISTA
# ------------------------------------------------
# Aquí NO estamos analizando canciones todavía,
# solo contexto del artista.

print("✅ Artista encontrado:", artista['name'])
print("🆔 ID del artista:", artist_id)

# Géneros asociados por Spotify (pueden ser múltiples)
print("🎼 Géneros:", artista['genres'])

# Popularidad:
# - Escala de 0 a 100
# - Calculada internamente por Spotify
# - Basada en reproducciones recientes y tendencia
print("🔥 Popularidad:", artista['popularity'])  
# Referencia común:
#   >70  → artista muy popular
#   >85  → artista global / mainstream

# Followers:
# - Número real de usuarios que siguen al artista
# - Métrica acumulada (no temporal)
print("👥 Seguidores:", artista['followers']['total'])
# Interpretación orientativa:
#   >1.000.000    → artista popular
#   >10.000.000   → superestrella
#   >50.000.000   → ícono global


✅ Artista encontrado: Shakira | ID: 0EmeFodog0BfCgMzAIvKQp
Géneros del artista: ['latin pop']
Popularidad: 88
Número total de seguidores: 40816211


# 3. Extracción de datos Spotify

In [4]:
# ────────────────────────────────────────────────
# FUNCIÓN SEGURA PARA HACER SEARCH
# ────────────────────────────────────────────────

# Esta función realiza búsquedas en Spotify de forma segura,
# manejando posibles límites de solicitudes (429) y errores inesperados.

def spotify_search_seguro(spotify, query, tipo, limite=10, reintentos=3):
    """
    Función que busca en Spotify de forma segura.
    Detecta el error 429 (rate limit) y espera si es necesario.
    
    Parámetros:
    - spotify: objeto de la API
    - query: texto a buscar
    - tipo: "artist" o "album"
    - limite: máximo resultados
    - reintentos: cuántas veces intentar si hay error 429
    """
    intento = 0  # Contador de intentos

    while intento < reintentos:
        try:
            # Llamada a Spotify
            return spotify.search(q=query, type=tipo, limit=limite)

        except SpotifyException as e:
            if e.http_status == 429:
                # Si nos dicen "espera", tomamos el tiempo de Retry-After
                espera = int(e.headers.get("Retry-After", 1))
                print(f"⏳ Demasiadas peticiones. Esperando {espera} segundos...")
                time.sleep(espera)
                intento += 1  # Sumamos intento
            else:
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            # Capturamos cualquier error inesperado
            print("❌ Error inesperado:", e)
            return None

    # Si agotamos todos los intentos
    print("⚠️ Demasiados intentos. Se omite esta búsqueda.")
    return None


In [ ]:
# Esta función obtiene todas las canciones de un álbum de Spotify de forma segura, manejando límites y errores.

def spotify_album_tracks_seguro(spotify, id_album, reintentos=3):
    """
    Obtiene de manera segura todas las canciones de un álbum de Spotify.

    Parámetros:
    - spotify: objeto de la API de Spotify
    - id_album: ID del álbum
    - reintentos: número de intentos si se recibe un error 429 (rate limit)

    Retorna:
    - Lista de tracks si la operación fue exitosa
    - None si ocurre un error no recuperable o se superan los reintentos
    """
    # Inicializa el contador de intentos para manejar posibles errores de límite de solicitudes (429)
    intento = 0

    # Bucle que permite reintentar la solicitud hasta un máximo definido
    while intento < reintentos:
        try:
            # Lista vacía donde se irán almacenando las canciones del álbum
            tracks = []
            # Configura la paginación de la API, Spotify devuelve hasta 50 items por petición
            offset = 0
            limit = 50  # máximo permitido por Spotify

            # Bucle para recorrer todas las páginas de canciones
            while True:
                # Llama a la API de Spotify para obtener un bloque de canciones
                response = spotify.album_tracks(id_album, limit=limit, offset=offset)
                # Añade las canciones obtenidas a la lista completa
                tracks.extend(response['items'])
                # Si no hay más páginas, termina el bucle
                if response['next'] is None:
                    break
                # Ajusta el offset para obtener la siguiente página de canciones
                offset += len(response['items'])

            # Devuelve la lista completa de canciones si todo fue exitoso
            return tracks

        except SpotifyException as e:
            # Captura errores específicos de la API de Spotify
            if e.http_status == 429:
                # Si es un error de rate limit
                # Obtiene el tiempo que indica Spotify para reintentar
                espera = int(e.headers.get("Retry-After", 1))
                # Informa al usuario y espera antes de reintentar
                print(f"⏳ Rate limit en álbum. Esperando {espera} segundos antes de reintentar...")
                time.sleep(espera)
                # Incrementa el contador de intentos
                intento += 1
            else:
                # Otros errores de Spotify se consideran irreparables
                print("❌ Error de Spotify:", e)
                return None

        except Exception as e:
            # Captura cualquier otro error inesperado
            print("❌ Error inesperado:", e)
            # Devuelve None para indicar fallo
            return None

    # Si se superan todos los intentos de reintento, se omite la operación
    print("⚠️ Demasiados intentos con este álbum. Se omite la operación.")
    return None


In [6]:
# ────────────────────────────────────────────────
# FUNCIONES DE NEGOCIO (LÓGICA DEL PROBLEMA)
# ────────────────────────────────────────────────

# La función busqueda_spotify sirve para buscar y recopilar canciones de Spotify

def busqueda_spotify(generos, año, spotify):
    todas_las_canciones = []          # Lista para almacenar todas las canciones encontradas
    albumes_ya_vistos = set()         # Conjunto para evitar álbumes duplicados

    print("Buscando canciones del año", año)

    for genero in generos:
        print("Género:", genero)

        # Busca artistas del género usando la función segura
        resultado_artistas = spotify_search_seguro(spotify, query=f"genre:{genero}", tipo="artist", limite=50)
        if resultado_artistas is None:
            continue  # Saltamos si hubo error
        artistas = resultado_artistas["artists"]["items"]

        for artista in artistas:
            nombre_artista = artista["name"]

            # Busca álbumes del artista en el año indicado
            busqueda = f"artist:{nombre_artista} year:{año}"
            resultado_albumes = spotify_search_seguro(spotify, query=busqueda, tipo="album", limite=50)
            if resultado_albumes is None:
                continue
            albumes = resultado_albumes["albums"]["items"]

            for album in albumes:
                id_album = album["id"]
                nombre_album = album["name"]

                if id_album in albumes_ya_vistos:
                    continue
                albumes_ya_vistos.add(id_album)

                # Obtiene canciones del álbum
                resultado_canciones = spotify.album_tracks(id_album)
                canciones = resultado_canciones["items"]

                for cancion in canciones:
                    info = {
                        "nombre": cancion["name"],
                        "artista": nombre_artista,
                        "album": nombre_album,
                        "genero": genero,
                        "año": año
                    }
                    todas_las_canciones.append(info)

        print("  → Canciones de este género añadidas\n")

    # Resumen final
    print("RESUMEN:")
    for genero in generos:
        contador = sum(1 for c in todas_las_canciones if c["genero"] == genero)
        print(f"- {genero}: {contador} canciones")
    print("Total de canciones encontradas:", len(todas_las_canciones))

    return pd.DataFrame(todas_las_canciones)


In [ ]:
# LLAMADA A LA FUNCION PARA EXTRAER DATOS DE SPOTIFY
genero = ["country","latin","jazz","rock"] # Lista de géneros musicales a analizar
año = 2018 # cambiamos este valor para cada año seleccionado
canciones_2018_df = busqueda_spotify(genero,año, spotify) # Ejecuta la búsqueda en Spotify y devuelve los resultados en un DataFrame

Buscando canciones del año 2018
Género: country


Your application has reached a rate/request limit. Retry will occur after: 30502 s


In [ ]:
canciones_2018_df  # Muestra el contenido completo del DataFrame


,nombre,artista,album,genero,año
0,"Twelfth night, Op. 42, No. 1",Sam Barber,Barber: An American Romantic,country,2012
1,"To be sung on the water, Op. 42, No. 2",Sam Barber,Barber: An American Romantic,country,2012
2,"The virgin martyrs, Op. 8, No. 1",Sam Barber,Barber: An American Romantic,country,2012
3,"Let down the bars, O Death, Op. 8, No. 2",Sam Barber,Barber: An American Romantic,country,2012
4,"Reincarnations, Op. 16: I. Mary Hynes",Sam Barber,Barber: An American Romantic,country,2012
...,...,...,...,...,...
13806,Long Progression,Red Hot Chili Peppers,Strange Man / Long Progression,rock,2012
13807,Magpies on Fire,Red Hot Chili Peppers,Magpies on Fire / Victorian Machinery,rock,2012
13808,Victorian Machinery,Red Hot Chili Peppers,Magpies on Fire / Victorian Machinery,rock,2012
13809,The Sunset Sleeps,Red Hot Chili Peppers,The Sunset Sleeps / Hometown Gypsy,rock,2012


In [ ]:
canciones_2018_df.head() # Muestra las primeras filas del DataFrame para inspeccionar la estructura

,nombre,artista,album,genero,año
0,"Twelfth night, Op. 42, No. 1",Sam Barber,Barber: An American Romantic,country,2012
1,"To be sung on the water, Op. 42, No. 2",Sam Barber,Barber: An American Romantic,country,2012
2,"The virgin martyrs, Op. 8, No. 1",Sam Barber,Barber: An American Romantic,country,2012
3,"Let down the bars, O Death, Op. 8, No. 2",Sam Barber,Barber: An American Romantic,country,2012
4,"Reincarnations, Op. 16: I. Mary Hynes",Sam Barber,Barber: An American Romantic,country,2012


In [ ]:
canciones_2018_df.info() # Muestra información estructural del DataFrame (columnas, tipos y nulos)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13811 entries, 0 to 13810
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   nombre   13811 non-null  object
 1   artista  13811 non-null  object
 2   album    13811 non-null  object
 3   genero   13811 non-null  object
 4   año      13811 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 539.6+ KB


In [ ]:
canciones_2018_df.isnull().sum() # Cuenta los valores nulos (NaN) por columna

nombre     0
artista    0
album      0
genero     0
año        0
dtype: int64

# 4. Credenciales y conexión LAST.FM

In [67]:
# --- COMPROBACIÓN DE LA CLAVE DE LAST.FM ---
# Recupera la API key desde el archivo .env
api_key_lastfm = os.getenv("API_KEY_LASTFM")

# Verifica que la clave exista antes de usarla
if not api_key_lastfm:
    print("⚠️ ADVERTENCIA: La clave de Last.fm no está cargada. Revisa tu archivo .env.")
else:
    # Confirmamos que la clave está disponible para futuras peticiones
    print("✅ Conexión con Last.fm inicializada.")

✅ Conexión con Last.fm inicializada.


In [68]:
# --- PRUEBA DE CONEXIÓN BÁSICA CON LAST.FM ---
if not api_key_lastfm:
    print("❌ Clave API de Last.fm no definida.")
else:
    try:
        # Hacemos una petición mínima para verificar la conexión usando el método 'chart.getTopArtists'
        respuesta = requests.get(
            "http://ws.audioscrobbler.com/2.0/",
            params={
                'method': 'chart.gettopartists',
                'api_key': api_key_lastfm,
                'format': 'json',
                'limit': 1  # Solo necesitamos 1 artista para probar
            }
        )
        respuesta.raise_for_status()
        print("✅ Conexión exitosa con Last.fm")
    except requests.exceptions.RequestException as e:
        print(f"❌ Error al conectar con Last.fm: {e}")

✅ Conexión exitosa con Last.fm


# 5. Extracción de datos LAST.FM

In [69]:
url_last_fm = ("http://ws.audioscrobbler.com/2.0/") # URL base de la API de Last.fm para realizar consultas

In [ ]:
def busqueda_info_artista(nombre_artista, api_key_lastfm):
    artista_codificado = quote(nombre_artista)                        # Codifica espacios/caracteres especiales para URL
    url_last_fm = "http://ws.audioscrobbler.com/2.0/"                # URL base de la API de Last.fm
    params_info = {                                                   # Parámetros de la consulta
        'method': 'artist.getinfo',                                   # Método API para info del artista
        'artist': artista_codificado,                                 # Nombre del artista codificado
        'api_key': api_key_lastfm,                                     # Clave de API
        'format': 'json'                                              # Formato de respuesta JSON
    }
    
    try:
        response = requests.get(url_last_fm, params=params_info, timeout=10)  # Llamada GET con timeout
        response.raise_for_status()                                     # Lanza error si HTTP status != 200
        data = response.json()                                          # Convierte respuesta a diccionario
        
        if "artist" in data:                                           # Verifica que se encontró el artista
            artista_info = data['artist']                              
            bio_summary = artista_info.get('bio', {}).get('summary', '').split('<a href')[0].strip()  # Extrae resumen sin enlaces
            return {
                'bio_resumen': bio_summary,                             # Resumen de biografía
                'listeners': int(artista_info.get('stats', {}).get('listeners', 0)),  # Oyentes
            }
        else:
            return {'consulta_exitosa': False, 'error_lastfm': "No encontrado en Last.fm"}  # Artista no encontrado
        
    except requests.exceptions.RequestException as e:                   # Captura errores de red/HTTP
        status_code = getattr(e.response, 'status_code', 'N/A')        # Obtiene código de error si existe
        return {'consulta_exitosa': False, 'error_lastfm': f"Error API ({status_code}): {e}"}
    except Exception as e:                                             # Captura errores de procesamiento
        return {'consulta_exitosa': False, 'error_lastfm': f"Error Procesamiento: {e}"}


if not api_key_lastfm:                                                # Verifica si la API key está configurada
    print("ERROR: La clave de la API de Last.fm no está configurada.")
else:
    artistas_unicos = canciones_2018_df['artista'].unique()          # Extrae artistas únicos
    print(f"\nTotal de artistas únicos a consultar en Last.fm: {len(artistas_unicos)}")
    
    artistas_df = pd.DataFrame(artistas_unicos, columns=['artista'])  # DataFrame temporal de artistas
    print("\nIniciando consultas a Last.fm...")
    
    resultados_lastfm_serie = artistas_df['artista'].apply(           # Aplica la función a cada artista
        busqueda_info_artista, 
        args=(api_key_lastfm,)                                        # Pasa la API key como argumento adicional
    )
    
    datos_lastfm_df = pd.json_normalize(resultados_lastfm_serie)       # Convierte Serie de diccionarios a DataFrame
    datos_lastfm_df.insert(0, 'artista', artistas_unicos)             # Añade columna 'artista' al inicio
    
    df_final = pd.merge(                                               # Une DataFrames principal y Last.fm
        canciones_2018_df,
        datos_lastfm_df,                                               # DataFrame con datos de Last.fm
        on='artista',                                                  # Clave de unión
        how='left'                                                     # Mantiene todas las canciones aunque falten datos
    )
    
    print("Consultas a Last.fm terminadas y datos unidos al DataFrame.")  # Mensaje final



Total de artistas únicos a consultar en Last.fm: 118

Iniciando consultas a Last.fm...
Consultas a Last.fm terminadas y datos unidos al DataFrame.


In [71]:
df_final # Muestra el contenido completo del DataFrame

,nombre,artista,album,genero,año,bio_resumen,listeners,consulta_exitosa,error_lastfm
0,"Twelfth night, Op. 42, No. 1",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
1,"To be sung on the water, Op. 42, No. 2",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
2,"The virgin martyrs, Op. 8, No. 1",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
3,"Let down the bars, O Death, Op. 8, No. 2",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
4,"Reincarnations, Op. 16: I. Mary Hynes",Sam Barber,Barber: An American Romantic,country,2012,"Sam Barber (born April 15, 2003) is an America...",274380.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...
13806,Long Progression,Red Hot Chili Peppers,Strange Man / Long Progression,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN
13807,Magpies on Fire,Red Hot Chili Peppers,Magpies on Fire / Victorian Machinery,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN
13808,Victorian Machinery,Red Hot Chili Peppers,Magpies on Fire / Victorian Machinery,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN
13809,The Sunset Sleeps,Red Hot Chili Peppers,The Sunset Sleeps / Hometown Gypsy,rock,2012,Red Hot Chili Peppers is an American rock band...,7104490.0,NaN,NaN


In [ ]:
# Guardar DataFrame en CSV
df_final.to_csv('canciones_2018_con_lastfm.csv', index=False, sep=';', encoding='utf-8-sig')
print("DataFrame guardado correctamente en 'canciones_2018_con_lastfm.csv'.")

DataFrame guardado correctamente en 'canciones_2012_con_lastfm.csv'.
